# Imports

In [1]:
import torch
import numpy as np

np.set_printoptions(precision=3)

# Introduction to PyTorch

## Creating Tensors

In [2]:
a = [1, 2, 3]
b = np.array([4, 5, 6], dtype=np.int32)

t_a = torch.tensor(a)
t_b = torch.tensor(b)

print(t_a)
print(t_b)

tensor([1, 2, 3])
tensor([4, 5, 6], dtype=torch.int32)


In [4]:
t_ones = torch.ones(2, 3)
print(t_ones.shape)
print(t_ones)

torch.Size([2, 3])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


In [5]:
rand_tensor = torch.rand(2, 3)
print(rand_tensor)

tensor([[0.0034, 0.7942, 0.1764],
        [0.6868, 0.8452, 0.3129]])


## Manipulating Data Types and Shape

In [6]:
t_a_new = t_a.to(torch.int64)
print(t_a_new.dtype)

torch.int64


### Transposing a Tensor

In [8]:
t = torch.rand(3, 5)
t_tr = torch.transpose(t, 0, 1)

print(t.shape, ' --> ', t_tr.shape)

torch.Size([3, 5])  -->  torch.Size([5, 3])


### Reshaping a Tensor

In [11]:
t = torch.zeros(30)
t_reshape = t.reshape(5, 6)
print(t.shape, ' --> ', t_reshape.shape)

torch.Size([30])  -->  torch.Size([5, 6])


### Removing Dimensions (In this case, dimensions that have a size of 1)

In [14]:
t = torch.zeros(1, 2, 1, 4, 1)
t_sqz = torch.squeeze(t, 2)

print(t.shape, ' --> ', t_sqz.shape)

torch.Size([1, 2, 1, 4, 1])  -->  torch.Size([1, 2, 4, 1])


## Applying Mathematical Operations to Tensors

In [16]:
torch.manual_seed(1)

t1 = 2 * torch.rand(5, 2) - 1 # torch.rand returns a tensor with rand nums from a uniform distribution in range of 0 to 1
t2 = torch.normal(mean=0, std=1, size=(5, 2))
t3 = torch.multiply(t1, t2)
print(t3)

tensor([[ 0.4426, -0.3114],
        [ 0.0660, -0.5970],
        [ 1.1249,  0.0150],
        [ 0.1569,  0.7107],
        [-0.0451, -0.0352]])


In [17]:
t4 = torch.mean(t1, axis=0)
print(t4)

tensor([-0.1373,  0.2028])


In [19]:
''' Matrix * Matrix Multiplication '''
t5 = torch.matmul(t1, torch.transpose(t2, 0, 1))
print(t5)

tensor([[ 0.1312,  0.3860, -0.6267, -1.0096, -0.2943],
        [ 0.1647, -0.5310,  0.2434,  0.8035,  0.1980],
        [-0.3855, -0.4422,  1.1399,  1.5558,  0.4781],
        [ 0.1822, -0.5771,  0.2585,  0.8676,  0.2132],
        [ 0.0330,  0.1084, -0.1692, -0.2771, -0.0804]])


In [20]:
t6 = torch.matmul(torch.transpose(t1, 0, 1), t2)
print(t6)

tensor([[ 1.7453,  0.3392],
        [-1.6038, -0.2180]])


In [21]:
norm_t1 = torch.linalg.norm(t1, ord=2, dim=1)
print(norm_t1)

tensor([0.6785, 0.5078, 1.1162, 0.5488, 0.1853])


In [25]:
''' verify torch.linalg.norm computes L2 Norm of t1 correctly '''
assert np.all(np.sqrt(np.sum(np.square(t1.numpy()),axis=1)) == norm_t1.numpy())